In [2]:
import csv
''' Depois de tratar o texto, e guardar os csv's em (letras.csv), (nutricao.csv)
    Podemos de fato começar a aplicar o método do simplex para determinar as melhores dietas'''

nomes = ['HENRIQUE','LUCAS','RAFAEL'] #nossos nomes

#nutrientes da que nossa dieta precisa satisfazer 
nutrientes_minimos = ['Caloria','Calcio','VitaminaA','Riboflavina','AcidoAscorbico'] 

dieta_prodx_por_letra = dict() #dicionario de letra para dieta e prodx
with open('letras.csv',newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dieta_prodx_por_letra[row['Letra']] = (float(row['Dieta']),float(row['ProdutoX']))

Vamos definir agora nosso problema de minimização!

Se gostaríamos de reduzir o custo, nossas variáveis serão todas $x_i$, onde $x_i$ significa quantos dólares gastamos no alimento $i$.
Como o médico nos passou quantidades mínimas para suprir de nutrientes, essas quantidades formarão nossa matriz de restrição.

Teremos algo do tipo:
$$
    x_1c_1 + x_2c_2 + ... + x_nc_n \geq c\\
    x_1p_1 + x_2p_2 + ... + x_np_n \geq p\\
    x_1v_1 + x_2v_2 + ... + x_nv_n \geq v\\
    x_1r_1 + x_2r_2 + ... + x_nr_n \geq r\\
    x_1a_1 + x_2a_2 + ... + x_na_n \geq a\\
$$

Onde $c_i$ é quanto de calórias o alimento $i$ dá por unidade de dolar gasto e $c$ é a quantidade mínima de calorias que o médico nos recomendou. (Fiz a mesma coisa para os outros nutrientes)

Como isso não está exatamente na forma padrão, teremos que multiplicar por -1 todas as linhas, de forma a tornar todo $\geq$ em $\leq$.

In [3]:
''' ideia 
M = [ [c1,c2,c3,...,cn],
      [p1,p2,p3,...,pn],
      ...
    ]
'''
alimentos = []
with open('nutricao.csv','r',newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        alimentos.append(row)

In [ ]:
mercadorias_disponiveis = ['Farinhadetrigo(enriquecida)','Leite evaporado (lata)','Queijo (cheddar)',
                     'Fígado (boi)','Espinafre','Batata-doce','Feijão verde. seco']

alimentos_disponiveis = []
for a in alimentos:
    if a['Mercadoria'] in mercadorias_disponiveis:
        alimentos_disponiveis.append(a)
alimentos_disponiveis

Depois de conseguir resultados estranhos, percebi que as dimensões passadas pelo médico e pelas tabelas estão em dimensões diferentes:
- Dimensão Dieta: **$10^2$ calorias, $10^{-2}$ gramas, $10^2$ unidades internacionais, $10^{-1}$ miligramas e miligrama.**
- Dimensões Produto X: **$10^3$ calorias/dólar, $10^{−1}$ gramas/dólar, $10^3$ unidades internacionais/dólar, $10^{−1}$ miligramas/dólar, miligramas/dólar.**

In [5]:
tabela_sem_prod_x = []
for nutriente in nutrientes_minimos:
    linha_nutri = [float(x[nutriente]) for x in alimentos]
    tabela_sem_prod_x.append(linha_nutri)


In [ ]:
tabela_loja = []
for nutriente in nutrientes_minimos:
    linha_nutri = [float(x[nutriente]) for x in alimentos_disponiveis]
    tabela_loja.append(linha_nutri)
tabela_loja

In [7]:
#com a matriz de restrição sem o produto X pronta, já estamos quase finalizados, basta descobrir quanto produto X nos dá e o B para cada nome!
from copy import deepcopy
import numpy as np
import scipy.optimize

conversao_dieta = [10**-1, 10**-2, 10**-1, 10**-1, 1] # para conversão da restrição
conversao_prod_x = [1,10**-1,1,10**-1,1] # para conversao do produto x

def matriz_todos_alimentos(nome:str):
    nome = nome.upper()
    m = deepcopy(tabela_sem_prod_x)
    for i in range(5): #são cinco nutrientes
        if i < len(nome) and nome[i] in dieta_prodx_por_letra:
            letra = nome[i]
            val = float(dieta_prodx_por_letra[letra][1]) * conversao_prod_x[i]
            m[i].append(val)
        else: 
            m[i].append(0.)
    return np.matrix(m)

def matriz_loja(nome:str):
    nome = nome.upper()
    m = deepcopy(tabela_loja)
    for i in range(5): #são cinco nutrientes
        if i < len(nome) and nome[i] in dieta_prodx_por_letra:
            letra = nome[i]
            val = float(dieta_prodx_por_letra[letra][1]) * conversao_prod_x[i]
            m[i].append(val)
        else: 
            m[i].append(0.)
    return np.matrix(m)



def restricoes(nome:str):
    nome = nome.upper()
    r = []
    for i in range(5):
        if i < len(nome) and nome[i] in dieta_prodx_por_letra:
            letra = nome[i]
            val = float(dieta_prodx_por_letra[letra][0]) * conversao_dieta[i]
            r.append(val)
        else:
            r.append(0.)
    return np.array(r)



def dieta(nome:str,method:str):
    m = -matriz_loja(nome)
    r = -restricoes(nome)
    c = np.ones(len(alimentos_disponiveis) + 1) # esse é o funcional linear que será minizado
    return scipy.optimize.linprog(c=c,A_ub=m,b_ub=r,method=method)

# Respostas letra A
Aqui temos os valores obtidos para cada pessoa
- nome
- LUCAS
- RAFAEL

In [ ]:
print(dieta('henrique','highs'))
ar = dieta('henrique','highs').x
ar.shape = (8,1)

matriz_loja('henrique')*ar

In [ ]:
print(dieta('lucas','highs'))
ar = dieta('lucas','highs').x
ar.shape = (8,1)

matriz_loja('lucas')*ar

In [ ]:
print(dieta('rafael','simplex'))
ar = dieta('rafael','simplex').x
ar.shape = (8,1)

matriz_loja('rafael')*ar

# Letra b
Quanto cada um estará disposto a pagar pela vitamina A? riboflavina pura?

Estariamos dispostos a pagar o mínimo usando qualquer alimento (ele estando presente ou não na loja).

In [ ]:

for nome in nomes:
    #restricao para vitamina A
    rest_vitA = -restricoes(nome)[2]
    vetor_coef_vitA = (-matriz_todos_alimentos(nome))[2]
    dieta_vitA = scipy.optimize.linprog(c = np.ones(len(alimentos) + 1), A_ub = vetor_coef_vitA, b_ub= rest_vitA, method='highs')
    print(f"Valor {nome} para vitamina A:",dieta_vitA, sep='\n')

    #restricao para riboflavina
    rest_ribo = -restricoes(nome)[3]
    vetor_coef_ribo = (-matriz_todos_alimentos(nome))[3]
    dieta_ribo = scipy.optimize.linprog(c = np.ones(len(alimentos) + 1), A_ub = vetor_coef_ribo, b_ub= rest_ribo, method='highs')
    print(f"Valor {nome} para riboflavina:",dieta_ribo,sep='\n')


### Valores encontrados letra b
- Henrique :
  - Vitamina A = 0.0006533101045296169
  - Riboflavina = 0.09251968503937008

- Lucas :
  - Vitamina A = 0.009037456445993032
  - Riboflavina = 0.01377952755905512

- Rafael :
  - Vitamina A = 0.006424216027874565
  - Riboflavina = 0.01377952755905512
